In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA, IncrementalPCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import lightgbm as lgb

In [4]:
dataset = pd.read_csv("dataset_clone.csv")
y=dataset.iloc[:,-1]
X=dataset.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)



In [3]:
clf = LogisticRegression(random_state=0, penalty='none').fit(X_train, y_train)

In [4]:
clf.score(X_test, y_test)

0.3082706766917293

In [5]:
clf2=LinearSVC(penalty='l2',random_state=0, tol=1e-5,max_iter=9000,C=1,loss='squared_hinge').fit(X_train,y_train)
#clf2 = make_pipeline(StandardScaler(),LinearSVC(penalty='l2',random_state=0, tol=1e-5,max_iter=9000,C=1,loss='squared_hinge')).fit(X_train,y_train)

In [6]:
clf2.score(X_test, y_test), clf2.score(X_train,y_train)

(0.3308270676691729, 1.0)

In [7]:
clf3 = KNeighborsClassifier(n_neighbors=199).fit(X_train, y_train)

In [8]:
clf3.score(X_test, y_test)

0.22556390977443608

In [9]:
clf3 = SVC(random_state=1, kernel='rbf', C=80, gamma=0.0001).fit(X_train,y_train)

In [10]:
clf3.score(X_test, y_test)

0.40601503759398494

In [11]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [12]:
gs=GridSearchCV(clf3,param_grid).fit(X,y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [13]:
gs.cv_results_

{'mean_fit_time': array([3.22590981, 2.90486374, 2.90879188, 3.14414272, 3.1916008 ,
        3.01643968, 3.20180283, 3.05305429, 3.19754109, 3.06712198,
        3.20482917, 3.05680599]),
 'std_fit_time': array([0.17841639, 0.00599297, 0.06740299, 0.48446613, 0.17261358,
        0.09387891, 0.17127592, 0.11273523, 0.17818782, 0.11086474,
        0.1749167 , 0.11442708]),
 'mean_score_time': array([0.54430785, 0.50200553, 0.56045299, 0.50750308, 0.50728784,
        0.50748372, 0.5088213 , 0.5071557 , 0.50738134, 0.50971842,
        0.50737395, 0.50945802]),
 'std_score_time': array([0.04003115, 0.00629903, 0.10834059, 0.01663082, 0.00454766,
        0.00358675, 0.00242535, 0.00164854, 0.00118989, 0.00347222,
        0.00183559, 0.00658875]),
 'param_C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dt

In [14]:
gs.best_estimator_,gs.best_params_

(SVC(C=100, gamma=0.0001, random_state=1),
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [15]:
param_grid = [
  {'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
 ]

gs=GridSearchCV(clf3,param_grid).fit(X,y)
gs.cv_results_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'mean_fit_time': array([3.28216395, 3.26196895, 4.42452836, 4.13522758, 3.0548821 ,
        3.48752589, 3.68803492, 3.81658397, 3.64238634]),
 'std_fit_time': array([0.18895024, 0.32531112, 1.14137695, 0.68690801, 0.1154186 ,
        0.92989758, 0.51116434, 0.32242091, 0.28673085]),
 'mean_score_time': array([0.50749831, 0.60030479, 0.69078894, 0.55778871, 0.51039839,
        0.59986639, 0.55756497, 0.64523478, 0.60189142]),
 'std_score_time': array([0.00134268, 0.11016675, 0.1272967 , 0.09428228, 0.00374328,
        0.12306794, 0.02960839, 0.07219875, 0.08498597]),
 'param_C': masked_array(data=[40, 40, 40, 100, 100, 100, 400, 400, 400],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[0.001, 0.0001, 1e-05, 0.001, 0.0001, 1e-05, 0.001,
                    0.0001, 1e-05],
              mask=[False, False, False, False, False, False, False, False

In [16]:
gs.best_estimator_, gs.best_params_, gs.best_score_

(SVC(C=100, gamma=0.0001, random_state=1),
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'},
 0.41046986721144024)

In [17]:
param_grid2= [
  {'C': [1,10,80,100,400,1000], 'loss': ['hinge','squared_hinge']},]
gs2=GridSearchCV(LinearSVC(), param_grid2).fit(X,y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [18]:
gs2.best_score_

0.34466292134831467

In [19]:
red=PCA(n_components=441)

In [20]:
X_t=red.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.30)
clf3 = SVC(random_state=1, kernel='rbf', C=80, gamma=0.0001).fit(X_train,y_train)
clf3.score(X_test, y_test)

0.48872180451127817

In [22]:
red2=KernelPCA()
X_t=red.fit_transform(X)

param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},]
gs=GridSearchCV(clf3,param_grid).fit(X_t,y)
gs.best_estimator_, gs.best_score_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


(SVC(C=100, gamma=0.0001, random_state=1), 0.41046986721144024)

In [23]:
red2=IncrementalPCA()
X_t=red.fit_transform(X)
clf4=SVC()
param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs=GridSearchCV(clf4,param_grid).fit(X_t,y)
gs.best_estimator_, gs.best_score_, gs.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


(SVC(C=100, gamma=0.0001),
 0.41046986721144024,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [24]:
red3=KernelPCA()
X_t=red.fit_transform(X)
clf5=SVC()
param_grid = [{'C': [40,100,400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs=GridSearchCV(clf5,param_grid).fit(X_t,y)
gs.best_estimator_, gs.best_score_, gs.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


(SVC(C=100, gamma=0.0001),
 0.41046986721144024,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [25]:
red4=PCA()
X_t=red.fit_transform(X)
sc=StandardScaler()
sc.fit_transform(X_t)
clf6=SVC()
param_grid = [{'C': [0.1, 40, 100, 400], 'gamma': [0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
              {'C': [0.1, 40, 100, 400], 'kernel': ['linear'], 'gamma': [0.001, 0.0001, 0.00001]},
              {'C': [0.1, 40, 100, 400], 'kernel': ['poly'], 'gamma': [0.001, 0.0001, 0.00001],},]

gs=GridSearchCV(clf6,param_grid).fit(X_t,y)
gs.best_estimator_, gs.best_score_, gs.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


(SVC(C=100, gamma=0.0001),
 0.41046986721144024,
 {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'})

In [26]:
red4=PCA()
X_t=red.fit_transform(X)
sc=StandardScaler()
sc.fit_transform(X_t)
clf6=LinearSVC(max_iter=5000)
param_grid = [{'C': [0.1, 40, 100, 400], 'penalty': ['l2'], 'loss': ['hinge','squared_hinge']},
              {'C': [0.1, 40, 100, 400], 'penalty': ['l1'], 'loss': ['hinge'], 'dual': ['False']},]
gs=GridSearchCV(clf6,param_grid).fit(X_t,y)
gs.best_estimator_, gs.best_score_, gs.best_params_

clf2=LinearSVC(penalty='l2',random_state=0, tol=1e-5,max_iter=9000,C=1,loss='squared_hinge').fit(X_train,y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py", line 233, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py", line 965, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "/usr/local/l

In [17]:
ensemble1=RandomForestClassifier().fit(X_train, y_train)
ensemble1.score(X_test, y_test)

0.47368421052631576

In [9]:
ensemble2=AdaBoostClassifier().fit(X_train, y_train)
ensemble2.score(X_test, y_test)

0.37593984962406013

In [49]:
ensemble3=GradientBoostingClassifier().fit(X_train, y_train)
ensemble3.score(X_test,y_test)

0.41353383458646614

In [88]:
ensemble4 = lgb.LGBMClassifier()

param_grid = {
    'num_leaves': [25,28,31,33,35],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'n_estimators': [10, 100, 1000, 10000]
}

gbm = GridSearchCV(ensemble4, param_grid, cv=3)
gbm.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'n_estimators': [10, 100, 1000, 10000],
                         'num_leaves': [25, 28, 31, 33, 35]})

In [94]:
gbm.best_estimator_, gbm.best_score_, gbm.best_index_, gbm.best_params_

(LGBMClassifier(learning_rate=0.01, num_leaves=25),
 0.4254394314360048,
 25,
 {'learning_rate': 0.01, 'n_estimators': 100, 'num_leaves': 25})

In [95]:
ensemble5 = lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [21,22,23,24,25,26],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100]
}
gbm2 = GridSearchCV(ensemble5, param_grid, cv=3)
gbm2.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.01, 0.1], 'n_estimators': [100],
                         'num_leaves': [21, 22, 23, 24, 25, 26]})

In [105]:
gbm2.best_estimator_, gbm2.best_score_, gbm2.best_index_, gbm2.best_params_

(LGBMClassifier(learning_rate=0.01, num_leaves=21),
 0.4254394314360048,
 0,
 {'learning_rate': 0.01, 'n_estimators': 100, 'num_leaves': 21})

In [101]:
gbm = lgb.LGBMClassifier(num_leaves=25,
                        learning_rate=0.01,
                        n_estimators=200,
                        max_depth=7)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['multi_error','multiclass'],
        early_stopping_rounds=70)

[1]	valid_0's multi_error: 0.699248	valid_0's multi_logloss: 1.38216
Training until validation scores don't improve for 70 rounds
[2]	valid_0's multi_error: 0.699248	valid_0's multi_logloss: 1.3801
[3]	valid_0's multi_error: 0.699248	valid_0's multi_logloss: 1.3778
[4]	valid_0's multi_error: 0.699248	valid_0's multi_logloss: 1.37606
[5]	valid_0's multi_error: 0.699248	valid_0's multi_logloss: 1.37427
[6]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.37198
[7]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.36998
[8]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.36865
[9]	valid_0's multi_error: 0.669173	valid_0's multi_logloss: 1.36674
[10]	valid_0's multi_error: 0.654135	valid_0's multi_logloss: 1.36598
[11]	valid_0's multi_error: 0.669173	valid_0's multi_logloss: 1.36409
[12]	valid_0's multi_error: 0.676692	valid_0's multi_logloss: 1.36255
[13]	valid_0's multi_error: 0.661654	valid_0's multi_logloss: 1.36161
[14]	valid_0's multi_error: 0.639098	val

[121]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31233
[122]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31182
[123]	valid_0's multi_error: 0.639098	valid_0's multi_logloss: 1.31242
[124]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31263
[125]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31306
[126]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31391
[127]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.3142
[128]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31402
[129]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31379
[130]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31364
[131]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31468
[132]	valid_0's multi_error: 0.631579	valid_0's multi_logloss: 1.31464
[133]	valid_0's multi_error: 0.62406	valid_0's multi_logloss: 1.31396
[134]	valid_0's multi_error: 0.62406	valid_0's multi_logloss: 1.31453
[135]	val

LGBMClassifier(learning_rate=0.01, max_depth=7, n_estimators=200, num_leaves=25)

In [107]:
ensemble6 = lgb.LGBMClassifier()
param_grid = {
    'num_leaves': [22,21,20,19,18],
    'learning_rate': [0.01,0.001],
    'n_estimators': [100],
    'max_depth': [7,8,9,10]
}
gbm3 = GridSearchCV(ensemble5, param_grid, cv=3)
gbm3.fit(X_train, y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=3, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.01, 0.001],
                         'max_depth': [7, 8, 9, 10], 'n_estimators': [100],
                         'num_leaves': [22, 21, 20, 19, 18]})

In [108]:
gbm3.best_estimator_, gbm3.best_score_, gbm3.best_index_, gbm3.best_params_

(LGBMClassifier(learning_rate=0.01, max_depth=9, num_leaves=22),
 0.4254394314360048,
 10,
 {'learning_rate': 0.01,
  'max_depth': 9,
  'n_estimators': 100,
  'num_leaves': 22})